# word2vec, fastTextで日本語学習済みモデルを使用する方法

In [1]:
#パッケージのインポート
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
#import torchvision
#from torchvision import models,transforms
import cv2 
import time
import math
from matplotlib import cm

import gensim
import torchtext


## word2vecの日本語学習済みモデルを使用する実装

In [2]:
#そのままではtorchtextで読み込めないので、gensimライブラリを使用してWord2vecのformatで保存し直します

from gensim.models import KeyedVectors

#一度gensimライブラリで読み込んでword2vecのformatで保存する
model=KeyedVectors.load_word2vec_format(
    "./data/entity_vector/entity_vector.model.bin",binary=True)

#保存(10分くらい)
#model.wv.save_word2vec_format("../7_nlp_sentiment_transformer/data/japanese_word2vec_vectors.vec")
model.save_word2vec_format("./data/japanese_word2vec_vectors.vec")

In [14]:
#torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors=Vectors(name="./data/japanese_word2vec_vectors.vec")

In [15]:
#単語ベクトルの中身を確認します
print("1単語を表現する次元数:",japanese_word2vec_vectors.dim)
print("単語数：",len(japanese_word2vec_vectors.itos))

1単語を表現する次元数: 200
単語数： 1015474


#### 前処理

In [10]:
import re

def preprocessing_text(text):
    #半角全角の統一
    #今回は無視

    #英語の小文字化
    #今回は無視
    #output=output.lower()

    #改行、半角スペース、全角スペースを削除
    text=re.sub("\r","",text)
    text=re.sub("\n","",text)
    text=re.sub("　","",text)
    text=re.sub(" ","",text)

    #数字文字の一律0化
    text=re.sub(r"[0-9 ０-９]","0",text) #数字
    #記号と数字の除去
    #今回は無視。半角記号・数字・英字
    #今回は無視。全角記号

    #特定の文字を正規表現で置換する
    #今回は無視

    return text

from janome.tokenizer import Tokenizer

j_t=Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text,wakati=True)]


def tokenizer_with_preprocessing(text):
    text=preprocessing_text(text) #前処理の正規化
    ret=tokenizer_janome(text) #Janomeの単語分割
    return ret

In [11]:
import torchtext.data as data

max_length=25

TEXT=torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
use_vocab=True,lower=True,include_lengths=True, batch_first=True,fix_length=max_length)

LABEL=data.Field(sequential=False,use_vocab=False)

In [12]:
train_ds,val_ds,test_ds=torchtext.data.TabularDataset.splits(path="./data/",
train="text_train.tsv",validation="text_val.tsv",test="text_test.tsv", format="tsv",
fields=[("Text",TEXT),("Label",LABEL)])

#### 続き

In [16]:
#ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds,vectors=japanese_word2vec_vectors,min_freq=1)

In [19]:
#ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors

#torch.Size([52, 200]) ???

torch.Size([52, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.3807,  0.4007, -1.6304,  ..., -2.2620, -0.5581,  0.9880],
        [-3.2000,  2.9676, -0.9007,  ...,  1.8192, -0.4641,  0.3200]])

In [20]:
#ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fcfac549690>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

In [25]:
#姫-女性+男性のベクトルがどれと似ているか確認
import torch.nn.functional as F

#姫-女性+男性
tensor_calc=TEXT.vocab.vectors[42]-TEXT.vocab.vectors[39]+TEXT.vocab.vectors[48]

#コサイン類似度を計算
#dim=0は0次元目で計算してくださいという指定
print("女王：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[40],dim=0))
print("王：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[46],dim=0))
print("王子：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[47],dim=0))
print("機械：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[45],dim=0))

女王： tensor(0.3840)
王： tensor(0.3669)
王子： tensor(0.5489)
機械： tensor(-0.0867)


In [23]:
TEXT.vocab.vectors[39].shape

torch.Size([200])

## fastTextの日本語学習済みモデルを使用する実装

In [26]:
#torchtextで単語ベクトルとして読み込みます
#word2vecとは異なり、すぐに読み込めます
from torchtext.vocab import Vectors

japanese_fasttext_vectors=Vectors(name="../7_nlp_sentiment_transformer/data/vector_neologd/model.vec")

#単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：",len(japanese_fasttext_vectors.itos))

100%|██████████| 351122/351122 [00:30<00:00, 11416.53it/s]


1単語を表現する次元数： 300
単語数： 351122


In [27]:
#ベクトル化したボキャブラリーを作成します
TEXT.build_vocab(train_ds,vectors=japanese_fasttext_vectors,min_freq=1)

#ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape) #52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

#ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

torch.Size([52, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fcf90445550>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

In [29]:
#姫-女性+男性のベクトルがどれと似ているか確認
import torch.nn.functional as F

#姫-女性+男性
tensor_calc=TEXT.vocab.vectors[42]-TEXT.vocab.vectors[39]+TEXT.vocab.vectors[48]

#コサイン類似度を計算
#dim=0は0次元目で計算してくださいという指定
print("女王：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[40],dim=0))
print("王：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[46],dim=0))
print("王子：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[47],dim=0))
print("機械：",F.cosine_similarity(tensor_calc,TEXT.vocab.vectors[45],dim=0))

女王： tensor(0.3650)
王： tensor(0.3461)
王子： tensor(0.5531)
機械： tensor(0.1078)
